# Práctico de Ayuda para Tarea 1

## Configuración inicial

Instalamos las librerías a utilizar

In [1]:
!pip install pyreclab --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 6.7 MB/s eta 0:00:00


In [2]:
!pip install psutil

In [3]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 47.1 MB/s eta 0:00:00


Importamos las librerías

In [4]:
import pandas as pd
import pyreclab
import numpy as np
import scipy.sparse as sparse
import implicit
from sklearn.metrics.pairwise import cosine_similarity
import psutil
import time
import math

%matplotlib inline

Cargamos los sets de datos

In [5]:
trainfile =  pd.read_csv('training_set.csv')
validfile =  pd.read_csv('validation_set.csv')
items_styles = pd.read_csv('items_styles.csv')

Antes de recomendar siempre hay que realizar un análisis de datos.

In [6]:
trainfile.head()

,userID,itemID,styleID,rating
0,401,32780,12224,3.0
1,7548,21688,9020,3.5
2,7663,1521,568,4.0
3,1357,3824,1417,3.5
4,1361,727,263,4.5


## Generar y entrenar modelo

Para este práctico se usará como ejemplo FunkSVD y ALS, pero recuerden realizar el análisis para todos los algoritmos pedidos en el enunciado.

### Preprocesamiento de los datos a formato sparse

In [8]:
user_items = {}
itemset = set()

for row in trainfile.itertuples():
    if row[1] not in user_items:
        user_items[row[1]] = []

    user_items[row[1]].append(row[2])
    itemset.add(row[2])

# Recordar que se tiene set con los itemID ordenados de los items
itemset = np.sort(list(itemset))

sparse_matrix = np.zeros((len(user_items), len(itemset)))

for i, items in enumerate(user_items.values()):
    sparse_matrix[i] = np.isin(itemset, items, assume_unique=True).astype(int)

matrix = sparse.csr_matrix(sparse_matrix.T)

user_ids = {key: i for i, key in enumerate(user_items.keys())}
user_item_matrix = matrix.T.tocsr()

### Entrenamiento de los modelos

In [7]:
svd = pyreclab.SVD(dataset='training_set.csv', dlmchar=b',', header=False, usercol=0, itemcol=1, ratingcol=3)
svd.train(factors=100, maxiter=100, lr=0.01, lamb=0.1)

In [9]:
model_als = implicit.als.AlternatingLeastSquares(factors=100, iterations=10)
model_als.fit(user_item_matrix)

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

## Obtener métricas de recomendación

### MAE Y RMSE

In [ ]:
predlist, mae, rmse = svd.test(input_file='validation_set.csv',
                               dlmchar=b',',
                               header=False,
                               usercol=0,
                               itemcol=1,
                               ratingcol=3)

print('MAE: {}\nRMSE: {}'.format(mae, rmse))

MAE: 0.4461956109027944
RMSE: 0.596232686353075


In [ ]:
svd.predict('401', '7324')

3.794656753540039

Tras identificar el mejor modelo, utilizar estos valores de predicción para completar el archivo *rating_template_fill.csv*

**IMPORTANTE:** Al igual que en la vida real, en el archivo se pide colocar un rating para items que no se encuentran en el dataset. Para esto, pueden utilizar el rating promedio de los items, del usuario correspondiente o del dataset.

### MAP y NDCG

In [ ]:
top_n = 10

recommendList, maprec, ndcg = svd.testrec(input_file='validation_set.csv',
                                          dlmchar=b',',
                                          header=False,
                                          usercol=0,
                                          itemcol=1,
                                          ratingcol=3,
                                          topn=top_n,
                                          relevance_threshold=2,
                                          includeRated=False)

print('MAP: {}\nNDCG@{}: {}'.format(maprec, top_n, ndcg))

MAP: 0.007197419291226195
NDCG@10: 0.005794055223542524


Estas métricas también pueden ser obtenidas directamente desde su definición oficial, implementadas como funciones. Esta manera de obtención se recomienda al utilizar modelos de Implicit

In [10]:
def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def mean_average_precision(rel_vector):
    return np.mean([average_precision(r) for r in rel_vector])

In [11]:
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.


def ndcg_at_k(rel_vector, k):
    idcg = dcg_at_k(sorted(rel_vector, reverse=True), k)

    if not idcg:
        return 0.
    return dcg_at_k(rel_vector, k) / idcg

In [12]:
def evaluate_model(model, user_items_test, n):
  mean_map = 0.
  mean_ndcg = 0.
  for u in user_items_test.keys():
    rec = model.recommend(u, user_item_matrix[u], n)[0]
    rel_vector = [np.isin(user_items_test[u], rec, assume_unique=True).astype(int)]
    mean_map += mean_average_precision(rel_vector)
    mean_ndcg += ndcg_at_k(rel_vector, n)

  mean_map /= len(user_items_test)
  mean_ndcg /= len(user_items_test)

  return mean_map, mean_ndcg

In [19]:
top_n = 10

user_item_valid = validfile.groupby('userID')['itemID'].apply(list).to_dict()
map, ndcg = evaluate_model(model_als, user_item_valid, top_n)

print('MAP: {}\nNDCG@{}: {}'.format(map, top_n, ndcg))

MAP: 0.0009530791788856305
NDCG@10: 0.0011730205278592375


### Recall

In [ ]:
user_id = 5401
ranking = [int(r) for r in svd.recommend(str(user_id), top_n, includeRated=False)]
als_ranking = list(model_als.recommend(userid=user_id, user_items=user_item_matrix[user_id], N=top_n)[0])
als_ranking = [itemset[r] for r in als_ranking]

print('FunkSVD: Recommendation for user {}: {}'.format(user_id, ranking))
print('ALS: Recommendation for user {}: {}'.format(user_id, als_ranking))

FunkSVD: Recommendation for user 5401: [11895, 35351, 32283, 17070, 36039, 12181, 47658, 66592, 32306, 40980]
ALS: Recommendation for user 5401: [1414, 5916, 8555, 26201, 10265, 36270, 18293, 515, 51917, 38552]


In [ ]:
user_ratings = validfile[validfile['userID'] == user_id]
average_rating = user_ratings['rating'].mean()
relevants_items = user_ratings['itemID'].tolist()

print('Relevants items for user {}: {}'.format(user_id, relevants_items))

Relevants items for user 5401: [17538, 21822, 20478, 5441, 19960]


In [ ]:
def recall_at_k(relevant_items, recommended_items, k):
    relevant_items = set(relevant_items)
    recommended_items = set(recommended_items[:k])
    intersection = relevant_items.intersection(recommended_items)
    recall = len(intersection) / len(relevant_items)
    return recall

In [ ]:
recall = svd.recall(str(user_id), ranking, relevance_threshold=2)
item_recall = recall_at_k(relevants_items, als_ranking, top_n)

print('FunkSVD recall: {}'.format(recall))
print('ALS recall: {}'.format(item_recall))

FunkSVD recall: 0.0
ALS recall: 0.0


In [ ]:
recall_score = 0
recommendations = {}

for user_id in validfile['userID'].unique():
  ranking = [int(r) for r in svd.recommend(str(user_id), top_n, includeRated=False)]
  recall_score += svd.recall(str(user_id), ranking)
  recommendations[user_id] = ranking

print('Recall del dataset: {}'.format(recall_score / len(validfile['userID'].unique())))

Recall del dataset: 0.0


Cuando identifiquen su mejor modelo, usan el diccionario con las listas de recomendación para completar el archivo *ranking_template_fill.json*

## Obtener métricas de diversidad y novedad

### Diversidad

In [ ]:
item_categories = items_styles.set_index('itemID')['styleID'].to_dict()
categorias_recomendadas = [item_categories[item] for item in ranking]

# Calcular la proporción de categorías únicas
categorias_unicas = len(set(categorias_recomendadas))
diversidad = categorias_unicas / len(ranking)
print('Diversidad: {}'.format(diversidad))

Diversidad: 0.9


### Novedad

Primero se obtiene la popularidad de cada item. Para esto utilizaremos el set de entrenamiento.

In [ ]:
item_counts = trainfile['itemID'].value_counts()
total_interacciones = len(trainfile)
items_popularity = item_counts / total_interacciones
items_popularity = items_popularity.to_dict()

In [ ]:
def calcular_novedad(recomendaciones_usuarios):
    num_usuarios = len(recomendaciones_usuarios)
    novedad_total = 0

    for usuario, recomendaciones in recomendaciones_usuarios.items():
        novedad_usuario = 0

        for item in recomendaciones:
            novedad_usuario += np.log(1 / items_popularity[item])

        novedad_total += novedad_usuario / len(recomendaciones)

    novedad_promedio = novedad_total / num_usuarios
    return novedad_promedio

In [ ]:
novedad = calcular_novedad(recommendations)
print('Novedad: {}'.format(novedad))

Novedad: 9.560081491731518


## Obtener métricas de entrenamiento

### Tiempo de Ejecución

In [ ]:
# Iniciar el temporizador
start_time = time.time()

# Entrenamiento del modelo
svd.train(factors=100, maxiter=100, lr=0.01, lamb=0.1)

# Detener el temporizador
end_time = time.time()

print(f"Tiempo de entrenamiento: {end_time - start_time} segundos")

Tiempo de entrenamiento: 4.55519437789917 segundos


### Memoria Utilizada

In [ ]:
# Obtener información de memoria antes del entrenamiento
mem_info_before = psutil.virtual_memory().used

# Entrenamiento del modelo
svd.train(factors=100, maxiter=100, lr=0.01, lamb=0.1)

# Obtener información de memoria después del entrenamiento
mem_info_after = psutil.virtual_memory().used

# Calcular el uso de memoria durante el entrenamiento
memory_used = (mem_info_after - mem_info_before) / (1024 ** 2)  # Convertir a MB

print(f"Memoria utilizada: {memory_used} MB")


Memoria utilizada: 16.671875 MB


### Uso de CPU

In [ ]:
# Obtener el uso de CPU antes del entrenamiento
cpu_usage_before = psutil.cpu_percent(interval=None)

# Entrenamiento del modelo
svd.train(factors=100, maxiter=100, lr=0.01, lamb=0.1)

# Obtener el uso de CPU después del entrenamiento
cpu_usage_after = psutil.cpu_percent(interval=None)

print(f"Uso de CPU: {cpu_usage_after - cpu_usage_before}%")

Uso de CPU: 31.6%
